In [1]:
#!pip install python-dotenv decorator==4.3 networkx
#!pip install pycoingecko
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import pprint


%matplotlib inline

In [2]:
load_dotenv('../.env')

True

In [3]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

In [4]:
coin_market = cg.get_coins_markets(vs_currency='usd')
coin_market_df= pd.DataFrame(coin_market, columns=['id', 'current_price', 'high_24h' , 'low_24h'])
coin_market_df.set_index('id', inplace=True)
coin_market_df

,current_price,high_24h,low_24h
id,,,
bitcoin,55171.000000,55555.000000,50569.000000
ethereum,3541.530000,3629.350000,3365.050000
cardano,2.180000,2.250000,2.120000
tether,1.000000,1.030000,0.994969
binancecoin,431.070000,445.260000,417.160000
...,...,...,...
mina-protocol,4.640000,4.870000,4.330000
ravencoin,0.115066,0.121996,0.104348
yearn-finance,31367.000000,32802.000000,30054.000000


In [5]:
btc_history = cg.get_coin_market_chart_by_id(id= 'bitcoin', vs_currency = 'usd', days = 'max')
btc_history_df = pd.DataFrame(btc_history)
btc_history_df

,prices,market_caps,total_volumes
0,"[1367107200000, 135.3]","[1367107200000, 1500517590]","[1367107200000, 0]"
1,"[1367193600000, 141.96]","[1367193600000, 1575032004.0]","[1367193600000, 0.0]"
2,"[1367280000000, 135.3]","[1367280000000, 1501657493.0]","[1367280000000, 0.0]"
3,"[1367366400000, 117.0]","[1367366400000, 1298951550.0]","[1367366400000, 0.0]"
4,"[1367452800000, 103.43]","[1367452800000, 1148667722.0]","[1367452800000, 0.0]"
...,...,...,...
3077,"[1633132800000, 48181.87627260008]","[1633132800000, 906929641888.8245]","[1633132800000, 42089383189.81192]"
3078,"[1633219200000, 47777.762019063]","[1633219200000, 900582483713.1118]","[1633219200000, 29742127387.956066]"
3079,"[1633305600000, 48282.97113857447]","[1633305600000, 910331395887.4816]","[1633305600000, 26034594561.963406]"
3080,"[1633392000000, 49260.232321130694]","[1633392000000, 927812140868.0522]","[1633392000000, 32691763060.319107]"


In [6]:
def extract_volume(x):
    return x[0]
def extract_price(x):
    return x[1]

btc_history_df["volume"] = btc_history_df['prices'].apply(extract_volume)
btc_history_df["price"] = btc_history_df['prices'].apply(extract_price)
btc_history_df["market_cap"] = btc_history_df['market_caps'].apply(extract_price)
btc_history_df["total_volume"] = btc_history_df['total_volumes'].apply(extract_price)

In [7]:
btc_history_df = btc_history_df[['volume', 'price',
       'market_cap', 'total_volume']]

In [8]:
btc_history_df

,volume,price,market_cap,total_volume
0,1367107200000,135.300000,1.500518e+09,0.000000e+00
1,1367193600000,141.960000,1.575032e+09,0.000000e+00
2,1367280000000,135.300000,1.501657e+09,0.000000e+00
3,1367366400000,117.000000,1.298952e+09,0.000000e+00
4,1367452800000,103.430000,1.148668e+09,0.000000e+00
...,...,...,...,...
3077,1633132800000,48181.876273,9.069296e+11,4.208938e+10
3078,1633219200000,47777.762019,9.005825e+11,2.974213e+10
3079,1633305600000,48282.971139,9.103314e+11,2.603459e+10
3080,1633392000000,49260.232321,9.278121e+11,3.269176e+10


In [9]:
eth_history = cg.get_coin_market_chart_by_id(id= 'ethereum', vs_currency = 'usd', days = 'max')
eth_history_df = pd.DataFrame(eth_history)
eth_history_df

,prices,market_caps,total_volumes
0,"[1438905600000, 2.83162]","[1438905600000, 0.0]","[1438905600000, 90622.0]"
1,"[1438992000000, 1.33075]","[1438992000000, 80339475.0]","[1438992000000, 368070.0]"
2,"[1439164800000, 0.6875857090838133]","[1439164800000, 41556309.21112176]","[1439164800000, 400464.0745435489]"
3,"[1439251200000, 1.067378879429946]","[1439251200000, 64539006.306272194]","[1439251200000, 1518998.2053373822]"
4,"[1439337600000, 1.256613175101658]","[1439337600000, 76013261.05415402]","[1439337600000, 2073893.3450314845]"
...,...,...,...
2247,"[1633132800000, 3305.107041443724]","[1633132800000, 388950236002.1381]","[1633132800000, 24368105704.246353]"
2248,"[1633219200000, 3393.9242492696653]","[1633219200000, 399455864987.66345]","[1633219200000, 21335051735.331825]"
2249,"[1633305600000, 3426.387080851377]","[1633305600000, 403971441903.55585]","[1633305600000, 18130363405.685154]"
2250,"[1633392000000, 3390.310407759615]","[1633392000000, 400955543856.1877]","[1633392000000, 19898468513.475533]"


In [10]:
eth_history_df["volume"] = eth_history_df['prices'].apply(extract_volume)
eth_history_df["price"] = eth_history_df['prices'].apply(extract_price)
eth_history_df["market_cap"] = eth_history_df['market_caps'].apply(extract_price)
eth_history_df["total_volume"] = eth_history_df['total_volumes'].apply(extract_price)

In [11]:
eth_history_df = eth_history_df[['volume', 'price',
       'market_cap', 'total_volume']]
eth_history_df.head()

,volume,price,market_cap,total_volume
0,1438905600000,2.831620,0.000000e+00,9.062200e+04
1,1438992000000,1.330750,8.033948e+07,3.680700e+05
2,1439164800000,0.687586,4.155631e+07,4.004641e+05
3,1439251200000,1.067379,6.453901e+07,1.518998e+06
4,1439337600000,1.256613,7.601326e+07,2.073893e+06


In [12]:
lte_history = cg.get_coin_market_chart_by_id(id= 'litecoin', vs_currency = 'usd', days = 'max')
lte_history_df = pd.DataFrame(lte_history)
lte_history_df

,prices,market_caps,total_volumes
0,"[1367107200000, 4.29983]","[1367107200000, 73773387]","[1367107200000, 0]"
1,"[1367193600000, 4.3594]","[1367193600000, 74936909.0]","[1367193600000, 0.0]"
2,"[1367280000000, 4.18295]","[1367280000000, 72037636.0]","[1367280000000, 0.0]"
3,"[1367366400000, 3.64914]","[1367366400000, 62957992.0]","[1367366400000, 0.0]"
4,"[1367452800000, 3.38879]","[1367452800000, 58565340.0]","[1367452800000, 0.0]"
...,...,...,...
3077,"[1633132800000, 166.03298444923087]","[1633132800000, 11077015406.904276]","[1633132800000, 2943230546.085889]"
3078,"[1633219200000, 169.65830733575754]","[1633219200000, 11659851024.782589]","[1633219200000, 2859712957.102274]"
3079,"[1633305600000, 170.96527037444142]","[1633305600000, 11749810607.616468]","[1633305600000, 2912326781.574752]"
3080,"[1633392000000, 167.54549258443683]","[1633392000000, 11506766114.666618]","[1633392000000, 2685181954.2209654]"


In [13]:
lte_history_df["volume"] = lte_history_df['prices'].apply(extract_volume)
lte_history_df["price"] = lte_history_df['prices'].apply(extract_price)
lte_history_df["market_cap"] = lte_history_df['market_caps'].apply(extract_price)
lte_history_df["total_volume"] = lte_history_df['total_volumes'].apply(extract_price)

In [14]:
lte_history_df = lte_history_df[['volume', 'price',
       'market_cap', 'total_volume']]
lte_history_df.head()

,volume,price,market_cap,total_volume
0,1367107200000,4.29983,73773387.0,0.0
1,1367193600000,4.35940,74936909.0,0.0
2,1367280000000,4.18295,72037636.0,0.0
3,1367366400000,3.64914,62957992.0,0.0
4,1367452800000,3.38879,58565340.0,0.0


In [15]:
link_history = cg.get_coin_market_chart_by_id(id= 'chainlink', vs_currency = 'usd', days = 'max')
link_history_df = pd.DataFrame(link_history)
link_history_df

,prices,market_caps,total_volumes
0,"[1510185600000, 0.22537665242356383]","[1510185600000, 78881828.34824735]","[1510185600000, 3190025.855165697]"
1,"[1510272000000, 0.189036422857878]","[1510272000000, 66162748.000257306]","[1510272000000, 2121175.2731304658]"
2,"[1510358400000, 0.18122267085020952]","[1510358400000, 63427934.797573335]","[1510358400000, 2229297.6966014686]"
3,"[1510444800000, 0.1700132017426718]","[1510444800000, 59504620.60993513]","[1510444800000, 5361145.800394995]"
4,"[1510531200000, 0.1806910270317246]","[1510531200000, 63241859.46110361]","[1510531200000, 2942234.064639167]"
...,...,...,...
1423,"[1633132800000, 26.310431364502723]","[1633132800000, 11985438391.491632]","[1633132800000, 1209726617.1253884]"
1424,"[1633219200000, 26.974247784337233]","[1633219200000, 12319787469.834383]","[1633219200000, 975093495.5204914]"
1425,"[1633305600000, 27.609012037186666]","[1633305600000, 12627009283.328642]","[1633305600000, 914842196.7789143]"
1426,"[1633392000000, 26.60528129414363]","[1633392000000, 12185133063.65021]","[1633392000000, 1010407961.509155]"


In [16]:
link_history_df["volume"] = link_history_df['prices'].apply(extract_volume)
link_history_df["price"] = link_history_df['prices'].apply(extract_price)
link_history_df["market_cap"] = link_history_df['market_caps'].apply(extract_price)
link_history_df["total_volume"] = link_history_df['total_volumes'].apply(extract_price)

In [17]:
link_history_df = link_history_df[['volume', 'price',
       'market_cap', 'total_volume']]
link_history_df.head()

,volume,price,market_cap,total_volume
0,1510185600000,0.225377,7.888183e+07,3.190026e+06
1,1510272000000,0.189036,6.616275e+07,2.121175e+06
2,1510358400000,0.181223,6.342793e+07,2.229298e+06
3,1510444800000,0.170013,5.950462e+07,5.361146e+06
4,1510531200000,0.180691,6.324186e+07,2.942234e+06


In [18]:
ada_history = cg.get_coin_market_chart_by_id(id= 'cardano', vs_currency = 'usd', days = 'max')
ada_history_df = pd.DataFrame(ada_history)
ada_history_df

,prices,market_caps,total_volumes
0,"[1508284800000, 0.02684535467621909]","[1508284800000, 696021404.3079604]","[1508284800000, 2351678.122306208]"
1,"[1508371200000, 0.026829950465728686]","[1508371200000, 695622018.2559936]","[1508371200000, 2815155.9695999967]"
2,"[1508457600000, 0.030299606216252807]","[1508457600000, 785580027.6424098]","[1508457600000, 8883473.082062634]"
3,"[1508544000000, 0.028587962639027336]","[1508544000000, 741202123.8797704]","[1508544000000, 5308856.917095766]"
4,"[1508630400000, 0.027796038407662104]","[1508630400000, 720669848.4724126]","[1508630400000, 2901876.1220860113]"
...,...,...,...
1445,"[1633132800000, 2.2467692592328996]","[1633132800000, 71980741780.63368]","[1633132800000, 2511823046.243126]"
1446,"[1633219200000, 2.2472190210882728]","[1633219200000, 72017933348.18193]","[1633219200000, 2194904017.547218]"
1447,"[1633305600000, 2.2588385813642846]","[1633305600000, 72529381282.70782]","[1633305600000, 1560814022.0556097]"
1448,"[1633392000000, 2.195095205424968]","[1633392000000, 70582726654.44083]","[1633392000000, 1630672169.7123742]"


In [19]:
ada_history_df["volume"] = ada_history_df['prices'].apply(extract_volume)
ada_history_df["price"] = ada_history_df['prices'].apply(extract_price)
ada_history_df["market_cap"] = ada_history_df['market_caps'].apply(extract_price)
ada_history_df["total_volume"] = ada_history_df['total_volumes'].apply(extract_price)

In [20]:
ada_history_df = ada_history_df[['volume', 'price',
       'market_cap', 'total_volume']]
ada_history_df.head()

,volume,price,market_cap,total_volume
0,1508284800000,0.026845,6.960214e+08,2.351678e+06
1,1508371200000,0.026830,6.956220e+08,2.815156e+06
2,1508457600000,0.030300,7.855800e+08,8.883473e+06
3,1508544000000,0.028588,7.412021e+08,5.308857e+06
4,1508630400000,0.027796,7.206698e+08,2.901876e+06


In [21]:
to_merge_dict = {
    'BTC': btc_history_df , 'ETH': eth_history_df,
    'LTE':lte_history_df,'LINK':link_history_df, 'ADA':ada_history_df }

In [22]:
#Coin Merged DataFrame
merged_coin_df = pd.concat(to_merge_dict.values(), keys=to_merge_dict.keys(), axis=1, join='inner')
merged_coin_df.head()

BTC                                               ETH            \
          volume   price    market_cap total_volume         volume     price   
0  1367107200000  135.30  1.500518e+09          0.0  1438905600000  2.831620   
1  1367193600000  141.96  1.575032e+09          0.0  1438992000000  1.330750   
2  1367280000000  135.30  1.501657e+09          0.0  1439164800000  0.687586   
3  1367366400000  117.00  1.298952e+09          0.0  1439251200000  1.067379   
4  1367452800000  103.43  1.148668e+09          0.0  1439337600000  1.256613   

                                         LTE                       \
     market_cap  total_volume         volume    price  market_cap   
0  0.000000e+00  9.062200e+04  1367107200000  4.29983  73773387.0   
1  8.033948e+07  3.680700e+05  1367193600000  4.35940  74936909.0   
2  4.155631e+07  4.004641e+05  1367280000000  4.18295  72037636.0   
3  6.453901e+07  1.518998e+06  1367366400000  3.64914  62957992.0   
4  7.601326e+07  2.073893e+06  1367452800000  3.38879  58565340.0   

                         LINK                                        \
  total_volume         volume     price    market_cap  total_volume   
0          0.0  1510185600000  0.225377  7.888183e+07  3.190026e+06   
1          0.0  1510272000000  0.189036  6.616275e+07  2.121175e+06   
2          0.0  1510358400000  0.181223  6.342793e+07  2.229298e+06   
3          0.0  1510444800000  0.170013  5.950462e+07  5.361146e+06   
4          0.0  1510531200000  0.180691  6.324186e+07  2.942234e+06   

             ADA                                        
          volume     price    market_cap  total_volume  
0  1508284800000  0.026845  6.960214e+08  2.351678e+06  
1  1508371200000  0.026830  6.956220e+08  2.815156e+06  
2  1508457600000  0.030300  7.855800e+08  8.883473e+06  
3  1508544000000  0.028588  7.412021e+08  5.308857e+06  
4  1508630400000  0.027796  7.206698e+08  2.901876e+06

In [23]:
merged_coin_df.to_csv(f'crypto_data.csv')

In [24]:
#Alpaca Keys
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [25]:
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [26]:
aapl_data = api.get_barset('AAPL',timeframe='1D', limit=1000,).df
aapl_data.head()

AAPL                                  
                             open    high     low   close    volume
time                                                               
2017-10-17 00:00:00-04:00  159.75  160.87  159.23  160.47  16059637
2017-10-18 00:00:00-04:00  160.42  160.71  159.60  159.77  13668418
2017-10-19 00:00:00-04:00  156.75  157.08  155.02  155.98  32669974
2017-10-20 00:00:00-04:00  156.61  157.75  155.96  156.25  18427186
2017-10-23 00:00:00-04:00  156.89  157.69  155.50  156.20  16418890

In [27]:
amzn_data = api.get_barset('AMZN',timeframe='1D', limit=1000,).df
amzn_data.head()

AMZN                                     
                              open      high      low    close   volume
time                                                                   
2017-10-17 00:00:00-04:00  1005.59  1011.470  1004.38  1008.99  1487100
2017-10-18 00:00:00-04:00  1009.27  1022.310   996.55   996.99  1714126
2017-10-19 00:00:00-04:00   990.33   991.050   980.24   986.60  1963364
2017-10-20 00:00:00-04:00   993.53   994.620   982.00   982.91  1406057
2017-10-23 00:00:00-04:00   986.73   986.775   962.50   966.34  2341953

In [28]:
ba_data = api.get_barset('BA',timeframe='1D', limit=1000,).df
ba_data.head()

BA                                   
                             open      high     low   close   volume
time                                                                
2017-10-17 00:00:00-04:00  256.02  259.0500  256.02  258.57  1792575
2017-10-18 00:00:00-04:00  258.47  260.7300  254.57  260.01  1342680
2017-10-19 00:00:00-04:00  258.98  259.1000  253.84  259.02  1453197
2017-10-20 00:00:00-04:00  260.36  264.8272  259.44  264.69  2860789
2017-10-23 00:00:00-04:00  265.13  265.9800  262.14  262.35  2385662

In [29]:
nvda_data = api.get_barset('NVDA',timeframe='1D', limit=1000,).df
nvda_data.head()

NVDA                                    
                             open    high       low   close    volume
time                                                                 
2017-10-17 00:00:00-04:00  197.09  198.90  196.3500  197.72  10112393
2017-10-18 00:00:00-04:00  198.10  198.14  193.3200  197.54  12459872
2017-10-19 00:00:00-04:00  194.44  198.00  192.4501  197.79  12283460
2017-10-20 00:00:00-04:00  198.49  199.59  196.5400  196.86   9450691
2017-10-23 00:00:00-04:00  197.70  198.85  195.8000  196.62   9054036

In [30]:
brk_data = api.get_barset('BRK.A',timeframe='1D', limit=1000,).df
brk_data.head()

BRK.A                                       
                                open       high        low     close volume
time                                                                       
2017-09-22 00:00:00-04:00  274165.03  274200.00  272000.00  273130.0    204
2017-09-25 00:00:00-04:00  273379.00  273860.66  271250.00  273670.0    183
2017-09-26 00:00:00-04:00  273700.00  274270.00  272600.00  272690.0    169
2017-09-27 00:00:00-04:00  274260.00  275745.00  273800.62  274940.0    159
2017-09-28 00:00:00-04:00  275870.00  275945.00  273825.00  274770.0    101

In [31]:
gs_data = api.get_barset('GS',timeframe='1D', limit=1000,).df
gs_data.head()

GS                                  
                             open    high      low   close   volume
time                                                               
2017-10-17 00:00:00-04:00  244.44  244.89  235.190  236.09  5860461
2017-10-18 00:00:00-04:00  236.87  242.65  236.870  242.02  3322005
2017-10-19 00:00:00-04:00  240.30  240.99  238.100  240.03  1917186
2017-10-20 00:00:00-04:00  242.95  245.25  240.670  244.74  2310872
2017-10-23 00:00:00-04:00  244.39  245.33  241.381  242.16  1475595

In [32]:
cop_data = api.get_barset('COP',timeframe='1D', limit=1000,).df
cop_data.head()

COP                                 
                            open    high      low  close   volume
time                                                             
2017-10-17 00:00:00-04:00  49.70  49.935  49.3200  49.73  2772828
2017-10-18 00:00:00-04:00  49.75  50.180  49.5550  49.71  5150559
2017-10-19 00:00:00-04:00  49.39  50.040  49.3000  49.72  3103441
2017-10-20 00:00:00-04:00  49.90  50.400  49.5601  50.25  4856606
2017-10-23 00:00:00-04:00  50.28  50.750  49.9300  50.27  3860560

In [33]:
gild_data = api.get_barset('GILD',timeframe='1D', limit=1000,).df
gild_data.head()

GILD                                 
                            open   high      low  close    volume
time                                                             
2017-10-17 00:00:00-04:00  80.00  80.71  79.5600  80.24   5209400
2017-10-18 00:00:00-04:00  80.26  80.84  79.7600  80.02   5240238
2017-10-19 00:00:00-04:00  81.53  82.47  80.5102  81.61  12294550
2017-10-20 00:00:00-04:00  81.83  81.95  80.4500  81.21   7203471
2017-10-23 00:00:00-04:00  81.25  81.32  80.0900  80.17   5181910

In [34]:
ccoi_data = api.get_barset('CCOI',timeframe='1D', limit=1000,).df
ccoi_data.head()

CCOI                             
                            open   high    low  close  volume
time                                                         
2017-10-16 00:00:00-04:00  49.85  50.25  49.25  49.50  141077
2017-10-17 00:00:00-04:00  49.65  50.00  49.15  49.90  151845
2017-10-18 00:00:00-04:00  50.05  50.20  49.60  49.70  202954
2017-10-19 00:00:00-04:00  49.65  50.05  47.80  48.41  128608
2017-10-20 00:00:00-04:00  48.89  50.50  47.88  49.65  204367

In [35]:
proc_data = api.get_barset('PG',timeframe='1D', limit=1000,).df
proc_data.head()

PG                               
                            open   high    low  close    volume
time                                                           
2017-10-17 00:00:00-04:00  93.04  93.09  92.40  92.80   4160781
2017-10-18 00:00:00-04:00  92.78  92.95  92.01  92.78   5978281
2017-10-19 00:00:00-04:00  91.70  91.99  91.14  91.63   6497132
2017-10-20 00:00:00-04:00  89.72  89.96  87.59  88.22  16520870
2017-10-23 00:00:00-04:00  88.63  88.86  86.87  87.29  10177891

In [36]:
# aapl_data.dropna(inplace=True)
# amzn_data.dropna(inplace=True)
# ba_data.dropna(inplace=True)
# nvda_data.dropna(inplace=True)
# brk_data.dropna(inplace=True)
# gs_data.dropna(inplace=True)
# cop_data.dropna(inplace=True) 
# gild_data.dropna(inplace=True)
# ccoi_data.dropna(inplace=True)
# proc_data.dropna(inplace=True)

In [37]:
merged_stock_df= pd.concat([
    aapl_data, amzn_data, ba_data, nvda_data, brk_data,
    gs_data, cop_data, gild_data, ccoi_data, proc_data], axis='columns', join='inner')
#merged_stock_df = pd.concat(to_merge_dict.values())
#merged_stock_df.dropna(inplace=True)
merged_stock_df.head()

AAPL                                       AMZN  \
                             open    high     low   close    volume     open   
time                                                                           
2017-10-17 00:00:00-04:00  159.75  160.87  159.23  160.47  16059637  1005.59   
2017-10-18 00:00:00-04:00  160.42  160.71  159.60  159.77  13668418  1009.27   
2017-10-19 00:00:00-04:00  156.75  157.08  155.02  155.98  32669974   990.33   
2017-10-20 00:00:00-04:00  156.61  157.75  155.96  156.25  18427186   993.53   
2017-10-23 00:00:00-04:00  156.89  157.69  155.50  156.20  16418890   986.73   

                                                                ...   CCOI  \
                               high      low    close   volume  ...   open   
time                                                            ...          
2017-10-17 00:00:00-04:00  1011.470  1004.38  1008.99  1487100  ...  49.65   
2017-10-18 00:00:00-04:00  1022.310   996.55   996.99  1714126  ...  50.05   
2017-10-19 00:00:00-04:00   991.050   980.24   986.60  1963364  ...  49.65   
2017-10-20 00:00:00-04:00   994.620   982.00   982.91  1406057  ...  48.89   
2017-10-23 00:00:00-04:00   986.775   962.50   966.34  2341953  ...  49.85   

                                                            PG                \
                            high     low  close  volume   open   high    low   
time                                                                           
2017-10-17 00:00:00-04:00  50.00  49.150  49.90  151845  93.04  93.09  92.40   
2017-10-18 00:00:00-04:00  50.20  49.600  49.70  202954  92.78  92.95  92.01   
2017-10-19 00:00:00-04:00  50.05  47.800  48.41  128608  91.70  91.99  91.14   
2017-10-20 00:00:00-04:00  50.50  47.880  49.65  204367  89.72  89.96  87.59   
2017-10-23 00:00:00-04:00  50.20  49.025  49.30  142925  88.63  88.86  86.87   

                                            
                           close    volume  
time                                        
2017-10-17 00:00:00-04:00  92.80   4160781  
2017-10-18 00:00:00-04:00  92.78   5978281  
2017-10-19 00:00:00-04:00  91.63   6497132  
2017-10-20 00:00:00-04:00  88.22  16520870  
2017-10-23 00:00:00-04:00  87.29  10177891  

[5 rows x 50 columns]

In [38]:
merged_stock_df.to_csv(f'stock_data.csv')

In [39]:
#Bond Data
#iSHares 20 year plus treasury
bond_data = api.get_barset(['TLT', 'VOOG', 'VGLT', 'VMBS', 'IWF', 'IGSB', 'GOVT'],timeframe='1D', limit=1000,).df


In [40]:
bond_data.dropna(inplace=True)
bond_data.head()

GOVT                                    IGSB  \
                            open    high    low  close    volume    open   
time                                                                       
2017-10-17 00:00:00-04:00  25.23  25.260  25.21  25.25  675884.0  52.590   
2017-10-18 00:00:00-04:00  25.20  25.210  25.17  25.20  507468.0  52.575   
2017-10-19 00:00:00-04:00  25.25  25.265  25.22  25.23  689206.0  52.635   
2017-10-20 00:00:00-04:00  25.13  25.170  25.13  25.15  651942.0  52.605   
2017-10-23 00:00:00-04:00  25.15  25.180  25.15  25.17  472528.0  52.625   

                                                             ...   VMBS  \
                             high     low   close    volume  ...   open   
time                                                         ...          
2017-10-17 00:00:00-04:00  52.611  52.580  52.599  621174.0  ...  52.84   
2017-10-18 00:00:00-04:00  52.615  52.569  52.615  446712.0  ...  52.79   
2017-10-19 00:00:00-04:00  52.645  52.605  52.630  529644.0  ...  52.77   
2017-10-20 00:00:00-04:00  52.614  52.595  52.605  545736.0  ...  52.65   
2017-10-23 00:00:00-04:00  52.640  52.601  52.635  856312.0  ...  52.68   

                                                                VOOG          \
                              high      low  close    volume    open    high   
time                                                                           
2017-10-17 00:00:00-04:00  52.8499  52.8000  52.82  441153.0  131.27  131.59   
2017-10-18 00:00:00-04:00  52.7900  52.7425  52.76  389960.0  131.88  131.88   
2017-10-19 00:00:00-04:00  52.8200  52.7300  52.73  424189.0  131.10  131.50   
2017-10-20 00:00:00-04:00  52.6892  52.6301  52.64  286452.0  132.00  132.08   
2017-10-23 00:00:00-04:00  52.7200  52.6700  52.72  262840.0  132.30  132.30   

                                                        
                                low     close   volume  
time                                                    
2017-10-17 00:00:00-04:00  131.2600  131.5800  29113.0  
2017-10-18 00:00:00-04:00  131.5301  131.6299  39530.0  
2017-10-19 00:00:00-04:00  130.8000  131.5000  32793.0  
2017-10-20 00:00:00-04:00  131.7100  131.9500  32071.0  
2017-10-23 00:00:00-04:00  131.2500  131.3300  29681.0  

[5 rows x 35 columns]

In [41]:
bond_data.to_csv(f'bond_data.csv')

In [42]:
#Longterm treasury shares
# vglt_data = api.get_barset('VGLT',timeframe='1D', limit=1000,).df
# vglt_data.head()

In [43]:
#Vanguard S&P 500 Growth
# voog_data = api.get_barset('VOOG',timeframe='1D', limit=1000,).df
# voog_data.head()

In [44]:
#Vanguard Mortgage Backed Securities
# vmbs_data = api.get_barset('VMBS',timeframe='1D', limit=1000,).df
# vmbs_data.head()

In [45]:
#iShares Russell 1000 Growth ETF
# iwf_data = api.get_barset('IWF',timeframe='1D', limit=1000,).df
# iwf_data.head()

In [46]:
#iShares 1-5 year Investment grade bond
# igsb_data = api.get_barset('IGSB',timeframe='1D', limit=1000,).df
# igsb_data.head()

In [47]:
#iShares US Treasury Bond ETF
# govt_data = api.get_barset('GOVT',timeframe='1D', limit=1000,).df
# govt_data.head()